# Tour of cudf

The RAPIDS cuDF library is a GPU DataFrame manipulation library based on Apache Arrow that accelerates loading, filtering, and manipulation of data for model training data preparation. The RAPIDS GPU DataFrame provides a pandas-like API that will be familiar to data scientists, so they can now build GPU-accelerated workflows more easily.

## Table of contents
* Loading data into a GPU DataFrame
    * Creating DataFrames in Python
    * Creating DataFrames from CSVs
    * Creating DataFrames from pandas
* Working with GPU DataFrames
    * Inspecting and slicing the data
    * Modifying data types
    * Manipulating the data
    * Sorting the data
    * Filter the data
    * Grouping, joining and merging data
    * One-hot encoding
    
Useful cheatsheet: https://rapids.ai/files/cheatsheet.pdf

# Loading data into a GPU DataFrame

## Creating DataFrames in Python

In [1]:
import os

import cudf
# import pandas as cudf to mimic this on CPU.
import pandas as pd
import numpy as np

In [2]:
gdf = cudf.DataFrame()
gdf['my_column'] = [1, 2, 3]
print(gdf)

gdf = cudf.DataFrame({'a': [1, 2], 'b': [3, 4]})
print(gdf)

  my_column
0         1
1         2
2         3
     b    a
0    3    1
1    4    2


## Creating DataFrames from CSVs

In [3]:
home = os.path.expanduser('~')
path = os.path.join(home, 'demo/docker/data/apartments.csv')
# Note: dtype detection is not yet supported.
names = ['city', 'zipcode', 'price_per_m2', 'year_built', 'population', 'median_income', 'date']
dtypes = ['category', 'int64', 'float64', 'float64', 'int64', 'int64', 'date']
# Don't use 'dtype' with pandas.
gdf = cudf.read_csv(path, names=names, dtype=dtypes, delimiter=';', skiprows=1, skipfooter=1)
print(gdf)

         city zipcode       price_per_m2         year_built population median_income                    date
 0 1265951705    2100  5444.022222000001        1985.111111       4332         26167 2018-09-06T00:00:00.000
 1 1265951705    2130             3768.0        1972.857143       5983         29579 2018-08-20T00:00:00.000
 2 1265951705    2140             2770.0            1977.75       3689         29447 2018-12-19T00:00:00.000
 3 1265951705    2160            4865.25             1990.0       3194         35167 2018-10-26T00:00:00.000
 4 1265951705    2170 3860.9375000000005 1972.9375000000002       5679         29153 2018-10-17T00:00:00.000
 5 1265951705    2180             3774.0 1986.6666670000002       6786         32550 2019-02-22T00:00:00.000
 6 1265951705    2200             3445.0             1963.5      10728         27063 2018-09-10T00:00:00.000
 7 1265951705    2210 2965.4888889999993 1979.5555560000003      11186         25047 2018-08-24T00:00:00.000
 8 1265951705    22

## Creating DataFrames from pandas

In [4]:
df = pd.read_csv(path, delimiter=';')

# Convert data types to ones supported by cudf.
df['city'] = df['city'].astype('category')
df['date'] = df['date'].astype("datetime64[ms]")

gdf = cudf.DataFrame.from_pandas(df) # Obviously, not applicable in pandas.
print(gdf)

    city zipcode price_per_m2         year_built population median_income                    date
 0 espoo    2100  5444.022222        1985.111111       4332         26167 2018-09-06T00:00:00.000
 1 espoo    2130       3768.0        1972.857143       5983         29579 2018-08-20T00:00:00.000
 2 espoo    2140       2770.0            1977.75       3689         29447 2018-12-19T00:00:00.000
 3 espoo    2160      4865.25             1990.0       3194         35167 2018-10-26T00:00:00.000
 4 espoo    2170    3860.9375          1972.9375       5679         29153 2018-10-17T00:00:00.000
 5 espoo    2180       3774.0 1986.6666670000002       6786         32550 2019-02-22T00:00:00.000
 6 espoo    2200       3445.0             1963.5      10728         27063 2018-09-10T00:00:00.000
 7 espoo    2210  2965.488889 1979.5555559999998      11186         25047 2018-08-24T00:00:00.000
 8 espoo    2230     4035.075             1992.3      20397         23074 2018-12-09T00:00:00.000
 9 espoo    2260    

# Working with GPU DataFrames

## Inspecting and slicing the data

In [5]:
print(gdf['population'])
print(gdf.head(3))

        
 0  4332
 1  5983
 2  3689
 3  3194
 4  5679
 5  6786
 6 10728
 7 11186
 8 20397
 9  3709
[83 more rows]
   city zipcode price_per_m2  year_built population median_income                    date
0 espoo    2100  5444.022222 1985.111111       4332         26167 2018-09-06T00:00:00.000
1 espoo    2130       3768.0 1972.857143       5983         29579 2018-08-20T00:00:00.000
2 espoo    2140       2770.0     1977.75       3689         29447 2018-12-19T00:00:00.000


In [6]:
print(gdf.loc[2:5, ['zipcode', 'year_built']])

  zipcode         year_built
2    2140            1977.75
3    2160             1990.0
4    2170          1972.9375
5    2180 1986.6666670000002


In [7]:
gdf['population'].mean(), gdf['population'].std()

(8014.397849462365, 4373.122998945762)

In [8]:
print(gdf['city'].value_counts())
print(gdf['city'].unique_count()) # unique() in pandas.

      
helsinki   65
espoo   28
2


In [9]:
train_fraction = 0.8
split_point = int(len(gdf)*train_fraction)

gdfs = {
    "train": gdf.loc[:split_point],
    "val": gdf.loc[split_point:]
}

print(gdfs)

{'val': <cudf.DataFrame ncols=7 nrows=19 >, 'train': <cudf.DataFrame ncols=7 nrows=75 >}


## Modifying data types

In [10]:
print(gdf.dtypes)

print('Median income dtype used to be:', gdf['median_income'].dtype)
gdf['median_income'] = gdf['median_income'].astype(np.float64)
print('Median income dtype is now:', gdf['median_income'].dtype)

city                   category
zipcode                   int64
price_per_m2            float64
year_built              float64
population                int64
median_income             int64
date             datetime64[ms]
dtype: object
Median income dtype used to be: int64
Median income dtype is now: float64


## Manipulating the data

In [11]:
def double_income(median_income):
    return 2*median_income

# Not in pandas.
gdf['median_income'] = gdf['median_income'].applymap(double_income)

print(gdf.head())

   city zipcode price_per_m2  year_built population median_income                    date
0 espoo    2100  5444.022222 1985.111111       4332       52334.0 2018-09-06T00:00:00.000
1 espoo    2130       3768.0 1972.857143       5983       59158.0 2018-08-20T00:00:00.000
2 espoo    2140       2770.0     1977.75       3689       58894.0 2018-12-19T00:00:00.000
3 espoo    2160      4865.25      1990.0       3194       70334.0 2018-10-26T00:00:00.000
4 espoo    2170    3860.9375   1972.9375       5679       58306.0 2018-10-17T00:00:00.000


In [12]:
gdf['my_feature'] = gdf['price_per_m2'] / gdf['population']
print(gdf.head())

   city zipcode price_per_m2  year_built population median_income                    date         my_feature
0 espoo    2100  5444.022222 1985.111111       4332       52334.0 2018-09-06T00:00:00.000 1.2566994972299168
1 espoo    2130       3768.0 1972.857143       5983       59158.0 2018-08-20T00:00:00.000 0.6297843891024569
2 espoo    2140       2770.0     1977.75       3689       58894.0 2018-12-19T00:00:00.000 0.7508809975603145
3 espoo    2160      4865.25      1990.0       3194       70334.0 2018-10-26T00:00:00.000  1.523246712586099
4 espoo    2170    3860.9375   1972.9375       5679       58306.0 2018-10-17T00:00:00.000 0.6798622116569819


In [13]:
# Not in pandas.
# Define input columns for the kernel
price_per_m2 = gdf['price_per_m2']
year_built = gdf['year_built']

def kernel(price_per_m2, year_built, out1, out2, kwarg1, kwarg2):
    for i, (x, y) in enumerate(zip(price_per_m2, year_built)):
       out1[i] = kwarg2 * x - kwarg1 * y
       out2[i] = y - kwarg1 * x
        
result = gdf.apply_rows(kernel,
                        incols=['price_per_m2', 'year_built'],
                        outcols=dict(out1=np.float64, out2=np.float64),
                        kwargs=dict(kwarg1=3, kwarg2=4))

print(result.head())

   city zipcode price_per_m2  year_built population median_income                    date ...                out2
0 espoo    2100  5444.022222 1985.111111       4332       52334.0 2018-09-06T00:00:00.000 ... -14346.955554999999
1 espoo    2130       3768.0 1972.857143       5983       59158.0 2018-08-20T00:00:00.000 ...        -9331.142857
2 espoo    2140       2770.0     1977.75       3689       58894.0 2018-12-19T00:00:00.000 ...            -6332.25
3 espoo    2160      4865.25      1990.0       3194       70334.0 2018-10-26T00:00:00.000 ...           -12605.75
4 espoo    2170    3860.9375   1972.9375       5679       58306.0 2018-10-17T00:00:00.000 ...           -9609.875
[2 more columns]


In [14]:
gdf.drop_column('date') # drop() in some pandas versions.
print(gdf.head())

   city zipcode price_per_m2  year_built population median_income         my_feature
0 espoo    2100  5444.022222 1985.111111       4332       52334.0 1.2566994972299168
1 espoo    2130       3768.0 1972.857143       5983       59158.0 0.6297843891024569
2 espoo    2140       2770.0     1977.75       3689       58894.0 0.7508809975603145
3 espoo    2160      4865.25      1990.0       3194       70334.0  1.523246712586099
4 espoo    2170    3860.9375   1972.9375       5679       58306.0 0.6798622116569819


## Sorting the data

In [15]:
gdf = gdf.sort_values(by='population', ascending=False)
gdf = gdf.reset_index()
print(gdf.head())

      city zipcode price_per_m2         year_built population median_income          my_feature
0 helsinki     940  1982.028571        1967.914286      25817       38172.0 0.07677222647867685
1    espoo    2230     4035.075             1992.3      20397       46148.0  0.1978268863068098
2 helsinki     530  5090.853659         1944.54878      18663       42582.0    0.27277788453089
3 helsinki     100  6284.152542        1935.067797      17868       51206.0 0.35169870953660176
4    espoo    2320       3003.0 1987.2222219999999      16322       46152.0 0.18398480578360496


In [16]:
print(gdf.nsmallest(n=3, columns=['population']))

      city zipcode price_per_m2         year_built population median_income         my_feature
92 helsinki     310       3971.0        1972.111111        896       46688.0  4.431919642857143
91 helsinki     130       7916.0             1911.4       1536       56220.0  5.153645833333333
90 helsinki     340  4497.333333 1973.3333329999998       1654       64768.0 2.7190648929866987


## Filtering the data

In [17]:
old = gdf.query("year_built < 1930")
print(old)

      city zipcode       price_per_m2         year_built population median_income         my_feature
31 helsinki     150 7727.5714290000005        1907.857143       9299       49734.0 0.8310110150553823
44 helsinki     140  7416.905659999999        1925.075472       7817       55194.0  0.948817405654343
46 helsinki     170             6310.5 1922.3823530000002       7391       54394.0 0.8538086862400216
91 helsinki     130             7916.0             1911.4       1536       56220.0  5.153645833333333


## Grouping, joining and merging data

In [18]:
# Difference to pandas: aggregated column names are prefixed with the aggregation function name.
# Also, 'city' becomes index in cudf but not in pandas.
grouped = gdf.groupby(['city']).agg({'zipcode': 'count', 'year_built': 'mean'})
print(grouped)

      city count_zipcode    mean_year_built
0    espoo            28 1986.1633418214285
1 helsinki            65 1969.2979250307694


In [19]:
pdf = pd.DataFrame({'city': ['helsinki', 'tampere'], 'feature1': [1, 2], 'feature2': [3, 4]})
pdf.city = pdf.city.astype('category')

left = grouped
right = cudf.DataFrame.from_pandas(pdf) # just 'pdf' with pandas.

In [20]:
print(left)
print(right)

      city count_zipcode    mean_year_built
0    espoo            28 1986.1633418214285
1 helsinki            65 1969.2979250307694
      city feature1 feature2
0 helsinki        1        3
1  tampere        2        4


In [21]:
# join() uses the index.
join_left = left.set_index('city')
join_right = right.set_index('city')

In [22]:
how = 'right' # Different join styles are supported.
joined = join_left.join(join_right, how=how)
joined = joined.to_pandas() # not used in pandas.
print(joined)

          feature1  feature2  count_zipcode  mean_year_built
helsinki         1         3             65      1969.297925
tampere          2         4             -1              NaN


In [23]:
# merge() uses some column. Only inner join is supported currently.
merged = left.merge(right, on=['city'])
print(merged)
merged['feature2'] = merged['feature2'].fillna(-1)
print(merged)

      city count_zipcode    mean_year_built feature1 feature2
0 helsinki            65 1969.2979250307694        1        3
1    espoo            28 1986.1633418214285                  
      city count_zipcode    mean_year_built feature1 feature2
0 helsinki            65 1969.2979250307694        1        3
1    espoo            28 1986.1633418214285                -1


## One-hot encoding

In [24]:
# get_dummies() in pandas.
gdf['city_codes'] = gdf.city.cat.codes
codes = gdf.city_codes.unique()
print(codes)
encoded = gdf.one_hot_encoding(column='city_codes', prefix='city_codes_dummy', cats=codes)

print(encoded.to_pandas())

      
0    0
1    1
        city  zipcode  price_per_m2   year_built  population  median_income  \
0   helsinki      940   1982.028571  1967.914286       25817        38172.0   
1      espoo     2230   4035.075000  1992.300000       20397        46148.0   
2   helsinki      530   5090.853659  1944.548780       18663        42582.0   
3   helsinki      100   6284.152542  1935.067797       17868        51206.0   
4      espoo     2320   3003.000000  1987.222222       16322        46152.0   
5   helsinki      200   5370.273585  1972.075472       14921        51890.0   
6      espoo     2760   3104.730769  1990.423077       14360        45784.0   
7   helsinki      420   3115.660000  1968.060000       13869        40204.0   
8   helsinki      560   4438.666667  1946.666667       13187        47068.0   
9   helsinki      700   3187.916667  1986.500000       13176        41880.0   
10  helsinki      180   7638.097222  1979.083333       13157        48798.0   
11  helsinki      500   5150.26